In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np

# Defining symbols and parameters
SYMBOLS = ["RELIANCE.NS", "TCS.NS", "INFY.NS", "HDFCBANK.NS", "ICICIBANK.NS"]
PERIOD = "3y"
TRADING_DAYS = 252
RF_ANNUAL = 0.06  # Indian risk-free estimate

# Downloading data from Yahoo Finance
data = yf.download(SYMBOLS, period=PERIOD, interval="1d", auto_adjust=False, group_by='ticker', threads=True)

# Organizing prices DataFrame
frames = []
for sym in SYMBOLS:
    df = data[sym].reset_index()
    df.columns = [c.lower().replace(" ", "_") for c in df.columns]
    df["symbol"] = sym
    frames.append(df[["date", "symbol", "open", "high", "low", "close", "adj_close", "volume"]])

prices = pd.concat(frames).dropna(subset=["date"]).copy()
prices["date"] = pd.to_datetime(prices["date"]).dt.date
prices = prices.sort_values(["symbol", "date"])
prices["ret"] = prices.groupby("symbol")["adj_close"].pct_change()

# Computing rolling risk metrics (volatility, Sharpe)
def compute_metrics(df, window=TRADING_DAYS, rf_annual=RF_ANNUAL):
    out = []
    for sym, g in df.groupby("symbol"):
        r = g["ret"].dropna()
        if r.size >= 2:
            rf_daily = (1 + rf_annual)**(1/window) - 1
            mu = r.tail(window).mean() if r.size >= window else r.mean()
            sigma = r.tail(window).std(ddof=1) if r.size >= window else r.std(ddof=1)
            vol_ann = float(sigma * np.sqrt(window)) if sigma and sigma > 0 else None
            sharpe = float(((mu - rf_daily) / sigma) * np.sqrt(window)) if sigma and sigma > 0 else None
            as_of = g["date"].max()
            out.append({
                "symbol": sym, "as_of_date": as_of, "period_days": min(window, r.size),
                "vol_annualized": vol_ann, "sharpe_annualized": sharpe
            })
    return pd.DataFrame(out)

metrics = compute_metrics(prices)

# Preparing correlation matrix
ret_pivot = prices.pivot(index="date", columns="symbol", values="ret").dropna(how="any")
corr = ret_pivot.corr().round(4)
corr.index.name = 'row_symbol'
corr.columns.name = 'col_symbol'
corr_long = corr.stack().reset_index(name='corr')

# Saving results for PowerBI
prices.to_csv("prices.csv", index=False)
metrics.to_csv("metrics.csv", index=False)
corr_long.to_csv("corr_long.csv", index=False)


[*********************100%***********************]  5 of 5 completed


In [ ]:
# Downloading datasets for PowerBI
from google.colab import files
files.download("prices.csv")
files.download("metrics.csv")
files.download("corr_long.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>